[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/SketchVideo-jupyter/blob/main/SketchVideo_jupyter.ipynb)

In [ ]:
%cd /content
!git clone --recursive -b v1.0 https://github.com/camenduru/SketchVideo
%cd /content/SketchVideo

!pip uninstall tensorflow -y
!pip install -q https://github.com/camenduru/wheels/releases/download/colab/detectron2-0.6-cp310-cp310-linux_x86_64.whl
!pip install -q https://github.com/camenduru/wheels/releases/download/colab/diffvg-0.0.1-cp310-cp310-linux_x86_64.whl
!pip install -q git+https://github.com/openai/CLIP
!pip install -q icecream wandb cssutils natsort

%cd /content/SketchVideo/layered-neural-atlases/thirdparty/RAFT
!wget https://huggingface.co/camenduru/SketchVideo/resolve/main/models.zip
!unzip models.zip

%cd /content/SketchVideo
!wget https://huggingface.co/camenduru/SketchVideo/resolve/main/DAVIS-2017-Unsupervised-trainval-Full-Resolution.zip
!unzip DAVIS-2017-Unsupervised-trainval-Full-Resolution.zip

In [ ]:
%cd /content/SketchVideo/data
!python process_dataset.py --mask_folder DAVIS/Annotations/Full-Resolution --image_folder DAVIS/JPEGImages/Full-Resolution --output_folder dataset --size 1080 1080

%cd /content/SketchVideo/layered-neural-atlases
!python preprocess_optical_flow.py --vid-path /content/SketchVideo/data/dataset/car-turn/imgs_crop --max_long_edge 224

%cd /content/SketchVideo/layered-neural-atlases
!python ../data/args.py --data_folder ../data/dataset/car-turn/imgs_crop --bg_folder ../data/dataset/car-turn/imgs_bg_crop  --evaluate_at 10000 --result_folder ../data/dataset/car-turn/results --resx 224 --resy 224 

%cd /content/SketchVideo/layered-neural-atlases
!python train.py config/config.json

In [ ]:
!mv /content/SketchVideo/data/dataset/car-turn/results/imgs_crop_02_07_2024__21_20_07_064766_/* /content/SketchVideo/data/dataset/car-turn/results/

In [ ]:
%cd /content/SketchVideo/CLIPavideo
!python run_object_sketching.py --target_file car-turn --focus foreground --data_folder /content/SketchVideo/data/dataset/car-turn --consist_param 3.0 --num_strokes 50 --num_iter 3001 --atlas_epoch '010000' --clip_model_name "RN101" --num_of_frames 60

In [ ]:
%cd /content/SketchVideo/CLIPavideo
!python run_object_sketching.py --target_file car-turn --focus background --data_folder /content/SketchVideo/data/dataset/car-turn --consist_param 0.15 --num_strokes 256 --num_iter 3001 --atlas_epoch '010000' --clip_model_name "ViT-B/32" --num_of_frames 60